In [399]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np
import statistics 

In [400]:
data_combined=pd.read_csv("covid_merge_house_2.csv")

In [401]:
housing_pred=pd.read_csv("final_results.csv")
reshaped_housing_pred = pd.melt(housing_pred, id_vars=['state'], value_vars=list(housing_pred)[:-1])

In [402]:
from datetime import datetime
from datetime import timedelta
reshaped_housing_pred['week_num']=reshaped_housing_pred['variable'].apply(lambda x: \
                (datetime.strptime(x, '%m/%d/%Y')+timedelta(days=6)).strftime('%V'))
reshaped_housing_pred['YM']=reshaped_housing_pred['variable'].apply(lambda x: \
                ((datetime.strptime(x, '%m/%d/%Y')+timedelta(days=6)).isoformat())[:7])
reshaped_housing_pred['total_value_estimate']=reshaped_housing_pred['value']

In [403]:
data_combined['week_num']=data_combined['covid_period_end'].apply(lambda x: \
                (datetime.strptime(x, '%Y-%m-%d')).strftime('%V'))
data_combined = data_combined.merge(reshaped_housing_pred[['state','week_num','total_value_estimate']]\
                                    ,on=['state','week_num'],how='inner')

In [404]:
data_combined['YM']=data_combined['covid_period_end'].apply(lambda x: x[:7])
data_for_clust=data_combined.groupby(['state','YM']).\
    agg({'new_cases':'sum','cum_cases':'max','total_value':'sum',\
         'total_value_estimate':'sum'}).reset_index()
data_for_clust['housing total value change%']=100*(data_for_clust['total_value']-data_for_clust['total_value_estimate'])/data_for_clust['total_value_estimate']

In [405]:
data_for_clust=data_for_clust[['state','YM','new_cases','cum_cases','housing total value change%']]
data_for_clust.head()

,state,YM,new_cases,cum_cases,housing total value change%
0,AK,2020-03,101.0,102,4.824520
1,AK,2020-04,235.0,337,-11.175549
2,AK,2020-05,104.0,441,-23.033093
3,AK,2020-06,586.0,1027,-20.042965
4,AK,2020-07,1839.0,2866,13.364975


In [406]:
scaler = StandardScaler()

def optimalK(data, nrefs=3, maxClusters=10):
    """
    Calculates KMeans optimal K using Gap Statistic from Tibshirani, Walther, Hastie
    Params:
        data: ndarry of shape (n_samples, n_features)
        nrefs: number of sample reference datasets to create
        maxClusters: Maximum number of clusters to test for
    Returns: (gaps, optimalK)
    """
    gaps = np.zeros((len(range(1, maxClusters)),))
    resultsdf = pd.DataFrame({'clusterCount':[], 'gap':[]})
    for gap_index, k in enumerate(range(1, maxClusters)):

        # Holder for reference dispersion results
        refDisps = np.zeros(nrefs)

        # For n references, generate random sample and perform kmeans getting resulting dispersion of each loop
        for i in range(nrefs):
            
            # Create new random reference set
            randomReference = np.random.random_sample(size=data.shape)
            
            # Fit to it
            km = KMeans(k)
            km.fit(randomReference)
            
            refDisp = km.inertia_
            refDisps[i] = refDisp

        # Fit cluster to original data and create dispersion
        km = KMeans(k)
        km.fit(data)
        
        origDisp = km.inertia_

        # Calculate gap statistic
        gap = np.log(np.mean(refDisps)) - np.log(origDisp)

        # Assign this loop's gap statistic to gaps
        gaps[gap_index] = gap
        
        resultsdf = resultsdf.append({'clusterCount':k, 'gap':gap}, ignore_index=True)

    return (gaps.argmax() + 1, resultsdf)  # Plus 1 because index of 0 means 1 cluster is optimal, index 2 = 3 clusters are optimal

In [407]:
lists_ofk=[]
for monthx in (['03','04','05','06','07','08','09','10']):
    temp=scaler.fit_transform(data_for_clust[data_for_clust['YM']=='2020-{}'.format(monthx)].values[:,2:4])
    temp_raw=data_for_clust[data_for_clust['YM']=='2020-{}'.format(monthx)].values[:,2:]
    temp_index=data_for_clust[data_for_clust['YM']=='2020-{}'.format(monthx)].values[:,:2]
    k, gapdf = optimalK(temp, nrefs=3, maxClusters=4)
    lists_ofk.append(k)
    print ('for month {}, optimal k is:{}'.format(monthx,k))

print('The mode is {}'.format(statistics.mode(lists_ofk)))

for month 03, optimal k is:3
for month 04, optimal k is:3
for month 05, optimal k is:3
for month 06, optimal k is:3
for month 07, optimal k is:2
for month 08, optimal k is:3
for month 09, optimal k is:3
for month 10, optimal k is:3
The mode is 3


In [408]:
def assign_level(df):
    level_col=[]
    for i in range(len(df)):
        label1=''
        for col in list(summary_temp)[1:]:
            if df.iloc[i][col]<low[col]: label1=label1+'L'
            elif df.iloc[i][col]<high[col]: label1=label1+'M'
            else: label1=label1+'H'
        level_col.append(label1)
    return level_col

def assign_level2(df):
    level_col=[]
    for i in range(len(df)):
        label2=''
        for col in list(summary_temp)[1:-1]:
            if df.iloc[i][col]<mean_[col]: label2=label2+'L'
            else: label2=label2+'H'
        level_col.append(label2)
    return level_col

In [409]:
temp_raw

array([[3466.0, 11710, 33.67060354405917],
       [20071.0, 171662, 27.652936870502934],
       [17667.0, 98422, 19.932699051280316],
       [13586.0, 230412, 21.259416405536843],
       [67043.0, 876257, 24.63864322478682],
       [15831.0, 84804, 34.99189789564001],
       [6243.0, 62830, 101.38231373225696],
       [1119.0, 16334, 15.857936591163686],
       [2786.0, 22942, 53.86013373357852],
       [53799.0, 752473, 16.40751959026274],
       [49915.0, 347468, 46.53728416758478],
       [1935.0, 14101, 6.879513707896433],
       [20790.0, 106698, 4.41518344570815],
       [12171.0, 52582, 32.153680261726784],
       [55104.0, 344787, 58.98036559251091],
       [29581.0, 148400, 33.361214734000896],
       [14764.0, 72616, 36.602665322560775],
       [18785.0, 88823, 17.060369937580717],
       [12574.0, 177726, 31.088367573576814],
       [12880.0, 142930, 54.587432195214845],
       [12135.0, 135463, 43.15958352101136],
       [653.0, 5913, 58.473003683148306],
       [24887.0, 1

In [410]:
lists_ofk = []
km_result_all=pd.DataFrame()

for monthx in (['03','04','05','06','07','08','09','10']):
    temp=scaler.fit_transform(data_for_clust[data_for_clust['YM']=='2020-{}'.format(monthx)].values[:,2:4])
    temp_raw=data_for_clust[data_for_clust['YM']=='2020-{}'.format(monthx)].values[:,2:]
    temp_index=data_for_clust[data_for_clust['YM']=='2020-{}'.format(monthx)].values[:,:2]
    kmeans = KMeans(n_clusters=3)
    kmeans.fit(temp)
    y_km = kmeans.fit_predict(temp)
    km_result=pd.DataFrame(np.c_[temp_index,temp_raw,y_km],columns=['state','month',\
                    'new_cases','cum_cases','housing total value change%','cluster'])
    km_result['new_cases']=km_result['new_cases'].astype('int')
    km_result['cum_cases']=km_result['cum_cases'].astype('int')
    km_result['housing total value change%']=km_result['housing total value change%'].astype('float')
    low=dict(km_result.quantile(0.33))
    mean_=dict(km_result.quantile(0.5))
    high=dict(km_result.quantile(0.67))
    summary_temp = km_result.groupby('cluster').agg({'new_cases':'mean',\
                                  'cum_cases':'mean'}).reset_index()
    summary_temp['label']=assign_level(summary_temp)
    summary_temp['label2']=assign_level2(summary_temp)
    km_result=km_result.merge(summary_temp[['cluster','label','label2']],on='cluster',how='inner')
    km_result_all=km_result_all.append(km_result)

In [411]:
km_result_all.groupby(['month','cluster','label'])['label'].count()

month    cluster  label
2020-03  0        MM       37
         1        HH        1
         2        HH        9
2020-04  0        MM       39
         1        HH        1
         2        HH        7
2020-05  0        MM       39
         1        HH        7
         2        HH        1
2020-06  0        MM       42
         1        HH        4
         2        HH        1
2020-07  0        MM       39
         1        HH        3
         2        HH        5
2020-08  0        HH        3
         1        MM       35
         2        HH        9
2020-09  0        HH       18
         1        LL       26
         2        HH        3
2020-10  0        HH       10
         1        LM       34
         2        HH        3
Name: label, dtype: int64

In [412]:
# km_result_all.to_csv('monthly_cluster_housing_price_change.csv',index=False)
km_result_all

,state,month,new_cases,cum_cases,housing total value change%,cluster,label,label2
0,AK,2020-03,101,102,4.824520,0,MM,LL
1,AL,2020-03,714,720,-6.277891,0,MM,LL
2,AR,2020-03,408,409,4.172275,0,MM,LL
3,AZ,2020-03,772,773,-12.527380,0,MM,LL
4,CO,2020-03,2059,2061,0.901185,0,MM,LL
...,...,...,...,...,...,...,...,...
42,NY,2020-10,28154,487626,27.045505,0,HH,HH
43,OH,2020-10,30216,180225,23.200151,0,HH,HH
44,PA,2020-10,25733,185670,44.226624,0,HH,HH
45,TN,2020-10,34403,222608,16.040469,0,HH,HH


In [413]:
out = km_result_all.merge(km_result_all,on='state',how='left')
out = out[out['month_x']>=out['month_y']].drop(out.columns[2:8],axis=1)
out.loc[out['month_x']==out['month_y'],'keep']=1
out.loc[out['label_y']=='HH','Cov_label']='High risk'
out['Cov_label']=out['Cov_label'].fillna('Medium to low risk')
out.columns=['state','month_label','month','new_cases','cum_cases','housing total value change%',\
             'cluster','label','label2','keep','Cov_label']
out

,state,month_label,month,new_cases,cum_cases,housing total value change%,cluster,label,label2,keep,Cov_label
0,AK,2020-03,2020-03,101,102,4.824520,0,MM,LL,1.0,Medium to low risk
8,AL,2020-03,2020-03,714,720,-6.277891,0,MM,LL,1.0,Medium to low risk
16,AR,2020-03,2020-03,408,409,4.172275,0,MM,LL,1.0,Medium to low risk
24,AZ,2020-03,2020-03,772,773,-12.527380,0,MM,LL,1.0,Medium to low risk
32,CO,2020-03,2020-03,2059,2061,0.901185,0,MM,LL,1.0,Medium to low risk
...,...,...,...,...,...,...,...,...,...,...,...
3003,WI,2020-10,2020-06,11720,30235,1.490662,0,MM,LH,NaN,Medium to low risk
3004,WI,2020-10,2020-07,21500,51735,41.608398,0,MM,LL,NaN,Medium to low risk
3005,WI,2020-10,2020-08,28032,79767,34.642384,1,MM,LL,NaN,Medium to low risk
3006,WI,2020-10,2020-09,40482,120249,50.331433,0,HH,HH,NaN,High risk


In [414]:
out.to_csv('monthly_cluster_housing_price_change.csv',index=False)

In [417]:
# data_combined[data_combined['state']=='TX']

,state,yyyy_ww,covid_period_end,covid_period_begin,new_cases,new_deaths,cum_cases,cum_deaths,new_cases_avg_2_wks,new_cases_avg_3_wks,...,new_deaths_avg_2_wks,new_deaths_avg_3_wks,new_deaths_avg_4_wks,period_begin,total_homes_sold,total_value,AVG_median_sales_price,week_num,total_value_estimate,YM
1307,TX,2020-09,2020-03-07,2020-03-01,8.0,0.0,19,0,7.5,5.666667,...,0.0,0.000000,0.00,2020-03-01,5422.0,1.430965e+09,263918.285319,10,1.299004e+09,2020-03
1308,TX,2020-10,2020-03-14,2020-03-08,50.0,0.0,69,0,29.0,21.666667,...,0.0,0.000000,0.00,2020-03-08,5432.0,1.406036e+09,258843.224411,11,1.420504e+09,2020-03
1309,TX,2020-11,2020-03-21,2020-03-15,478.0,5.0,547,5,264.0,178.666667,...,2.5,1.666667,1.25,2020-03-15,5481.0,1.441080e+09,262922.741471,12,1.389859e+09,2020-03
1310,TX,2020-12,2020-03-28,2020-03-22,1834.0,25.0,2381,30,1156.0,787.333333,...,15.0,10.000000,7.50,2020-03-22,5734.0,1.537039e+09,268056.999651,13,1.864477e+09,2020-03
1311,TX,2020-13,2020-04-04,2020-03-29,4185.0,94.0,6566,124,3009.5,2165.666667,...,59.5,41.333333,31.00,2020-03-29,6112.0,1.613241e+09,263946.554319,14,1.335531e+09,2020-04
1312,TX,2020-14,2020-04-11,2020-04-05,6642.0,141.0,13208,265,5413.5,4220.333333,...,117.5,86.666667,66.25,2020-04-05,4163.0,1.110425e+09,266736.687485,15,1.438118e+09,2020-04
1313,TX,2020-15,2020-04-18,2020-04-12,5719.0,222.0,18927,487,6180.5,5515.333333,...,181.5,152.333333,120.50,2020-04-12,5051.0,1.353936e+09,268052.999208,16,1.541807e+09,2020-04
1314,TX,2020-16,2020-04-25,2020-04-19,5567.0,175.0,24494,662,5643.0,5976.000000,...,198.5,179.333333,158.00,2020-04-19,4561.0,1.201868e+09,263509.702697,17,1.676430e+09,2020-04
1315,TX,2020-17,2020-05-02,2020-04-26,6939.0,213.0,31433,875,6253.0,6075.000000,...,194.0,203.333333,187.75,2020-04-26,6061.0,1.605761e+09,264933.266953,18,1.933964e+09,2020-05
1316,TX,2020-18,2020-05-09,2020-05-03,7416.0,211.0,38849,1086,7177.5,6640.666667,...,212.0,199.666667,205.25,2020-05-03,3959.0,1.004303e+09,253676.036120,19,1.541709e+09,2020-05


In [418]:
data_for_clust=data_combined.groupby(['state','YM']).\
    agg({'new_cases':'sum','cum_cases':'max','total_value':'sum',\
         'total_value_estimate':'sum'}).reset_index()
data_for_clust['housing total value change%']=100*(data_for_clust['total_value']-data_for_clust['total_value_estimate'])/data_for_clust['total_value_estimate']

In [420]:
# reshaped_housing_pred[reshaped_housing_pred['state']=='TX']

,state,variable,value,week_num,YM,total_value_estimate
40,TX,3/2/2020,1.299004e+09,10,2020-03,1.299004e+09
87,TX,3/9/2020,1.420504e+09,11,2020-03,1.420504e+09
134,TX,3/16/2020,1.389859e+09,12,2020-03,1.389859e+09
181,TX,3/23/2020,1.864477e+09,13,2020-03,1.864477e+09
228,TX,3/30/2020,1.335531e+09,14,2020-04,1.335531e+09
275,TX,4/6/2020,1.438118e+09,15,2020-04,1.438118e+09
322,TX,4/13/2020,1.541807e+09,16,2020-04,1.541807e+09
369,TX,4/20/2020,1.676430e+09,17,2020-04,1.676430e+09
416,TX,4/27/2020,1.933964e+09,18,2020-05,1.933964e+09
463,TX,5/4/2020,1.541709e+09,19,2020-05,1.541709e+09


In [425]:
temp = out[out['month_label']==out['month']].groupby(['month_label','Cov_label'])\
.agg({'new_cases':'mean','cum_cases':'mean'}).reset_index()
round(temp.groupby('Cov_label').agg({'new_cases':['mean','std'],'cum_cases':['mean','std']}),2)

new_cases            cum_cases           
                        mean       std       mean        std
Cov_label                                                   
High risk           63308.46  34841.93  209889.96  128930.00
Medium to low risk  10327.31   5395.83   38686.00   29210.37